#                              CREDIT ANALYTICS

## Importing libraries/packages

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

## CREATE CONNECTION TO BIGQUERY

In [2]:
KEY_PATH = '/work/datapro-405709-6e4eacf25d80.json'  # replace with your key path
PROJECT_ID = 'datapro-405709'   # replace with your project id

#Credentials and Client
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(KEY_PATH)
client = bigquery.Client(credentials=credentials, project=PROJECT_ID)
# Display the client details
client

## READ AND LOAD DATA FROM BIGQUERY

In [3]:
query = "SELECT * FROM `Credit_Business.Disbursements`"
query_job = client.query(query)

## CREATE PANDAS DATA FRAME 

In [4]:
# Retry the dataframe creation
df = query_job.to_dataframe()

## DATA WRANGLING

In [5]:
import db_dtypes
# Retry the dataframe creation
df = query_job.to_dataframe()

#SETTING PANDAS DISPLAY OPTIONS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.2f}'.format

#DROP COLUMNS
columns_to_drop = ['Branch_Code', 'Account_Number', 'Account_Name', 'S_N', 'GROUP___CUSTOMER_NUMBER']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# CONVERTING DATATYPES
df = df.astype({
    'Disbursed_Date': 'datetime64',
    'Disbursed_By': 'string',
    'Approved_By': 'string',
    'Branch_Name': 'string',
    'Insurance_company': 'string',
    'Sub_Sector': 'string',
    'Birth_date': 'datetime64',
    'Due_days': 'int64',
    'Interest_Rate': 'float64',
    'Principal_Outstanding': 'float64',
    'Interest_Outstanding': 'float64',
    'Grace_Period': 'string',
    'Loan_Cycle': 'int64',
    'Term': 'string',
    'Gender': 'string',
    'Product': 'string',
    'Sector': 'string',
    'Disbursed___Amount': 'float64',   
})

#DATAFRAME HEAD AND TAIL
#df.head(10)
#df.tail(10)

In [6]:
#DATA FRAME COLUMNS
#print(df.columns)

In [7]:
branches = df['Branch_Name'].unique()
Branch_Names=branches
#print(branches)

#                              Credit Sales

### Select Dates

In [8]:

from dateutil.parser import parse as _deepnote_parse
StartDate = _deepnote_parse('2023-01-10T00:00:00.000Z').date()


In [9]:

from dateutil.parser import parse as _deepnote_parse
Enddate = _deepnote_parse('2023-11-26T21:00:00.000Z').date()


## Company Overview

In [10]:
# Start and end dates conversion to datetime
StartDate = pd.to_datetime(StartDate)
Enddate = pd.to_datetime(Enddate)

# Filtering dataframe according to date range
df_date_range = df[(df['Disbursed_Date'] >= StartDate) & (df['Disbursed_Date'] <= Enddate)]

# Calculation of total amount disbursed and number of disbursements at end date
end_total_amount_disbursed = df_date_range['Disbursed___Amount'].sum()
end_total_rows = len(df_date_range)

# Calculation of total amount disbursed and number of disbursements at start date
start_df = df[df['Disbursed_Date'] < StartDate]
start_total_amount_disbursed = start_df['Disbursed___Amount'].sum()
start_total_rows = len(start_df)

# Calculating the change amount and change number of disbursements from start date to end date
change_amount_disbursed = end_total_amount_disbursed - start_total_amount_disbursed
change_disbursements = end_total_rows - start_total_rows

# Creating a new dataframe with all the information
disbursement_change = pd.DataFrame({
    'Previous Amounts': [start_total_amount_disbursed],
    'Cumulative Amount': [end_total_amount_disbursed],
    'Change in Amounts': [change_amount_disbursed],
    'Previous No': [start_total_rows],
    'Cummulative No': [end_total_rows],
    'Change in No': [change_disbursements],
})

# Applying formatting style and color formatting to the dataframe subset
disbursement_change.style.set_table_styles([{"selector": "", "props": [("border", "2px solid green"), ("width", "100%")]}]).format({
    'Previous Amounts': "{:,.2f}",
    'Cumulative Amount': "{:,.2f}",
    'Change in Amounts': "{:,.2f}",
    'Previous No': "{:,.0f}",
    'Cummulative No': "{:,.0f}",
    'Change in No': "{:,.0f}"
})

,Previous Amounts,Cumulative Amount,Change in Amounts,Previous No,Cummulative No,Change in No
0,"4,210,976,797.99","270,716,759,877.03","266,505,783,079.04","1,076","90,523","89,447"


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=34c99b87-7747-401d-b203-ddc5eb2646f3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>